# Курсовой проект по курсу Введение в обработку естественного языка

# Создание модели для чат-бота в Telegram

## Установка библиотек и модулей

In [ ]:
!pip install pymorphy2
!pip install stop_words
!pip install annoy

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 7.6 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=e8ea1b13d91c5267b4890f7b508f9883a6ecc993e7f23d637a43d3d9418ec564
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 646 kB 5.4 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391666 sha256=ebcb363aa8a30393be1b3a32984e27cab418f07bc374609d33916a389fb51f81
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [ ]:
import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

## Подключение google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/115_NLP/telegram_bot/data/'

## Предподготовка ответов (в Colab код выполняется один раз)

In [ ]:
assert True

#Small preprocess of the answers

question = None
written = False

with open(data_path + "prepared_answers.txt", "w") as fout:
    with open(data_path + "Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

## Модель

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return ' '.join(spls)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [ ]:
assert True

# Preprocess for models fitting

sentences = []

c = 0

with open(data_path + 'Otvety.txt', "r", encoding='utf8') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
sentences = [i for i in sentences if len(i) > 2]

In [ ]:
# Описание параметров Word2Vec https://radimrehurek.com/gensim/models/word2vec.html
modelW2V = Word2Vec(sentences=sentences,
                    size=300, # размер вектора слов
                    min_count=1, # минимальное количество слов, которые следует учитывать при обучении модели
                    window=3, # контекстное окно 3 слова
                    sg = 1, # алгоритм обучения skip-gram, так как корпус текста меньше 100 млн слов. По умолчанию - CBOW (0)
                    hs = 0, # если параметр 0 и negative, то применяется Negative Sampling
                    negative = 9, # Negative Sampling для уменьшения вычислительных затрат на обучение (3 контекстных слова + 6 слов, не совпадающих с контекстом)
                    sample = 1e-5, # субсэмплирование подразумевает избавление от слишком повторяющихся слов, рекомендуемый параметр от 1e-3 до 1е-5
                    workers= 1, # количество разделов во время обученияи и рабочих (по умолчанию равно 3)
                    seed = 13  # зерно для генератора случайных чисел. Для детерминировано-воспроизводимого прогона должно быть workers= 1
                    )

In [ ]:
modelFT = FastText(sentences=sentences,
                   size=300, 
                   min_count=1,
                   window=3, 
                   sg = 1, 
                   hs = 0, 
                   negative = 9, 
                   sample = 1e-5, 
                   workers= 1, 
                   seed = 13
                   )

In [ ]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open(data_path + "prepared_answers.txt", "r", encoding='utf8') as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += 1
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 200000:
            break

w2v_index.build(10)
ft_index.build(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


True

In [ ]:
modelW2V.save(data_path + 'modelw2v.bin')

In [ ]:
modelFT.save(data_path + 'modelft.bin')

In [ ]:
w2v_index.save(data_path + 'w2v_index.bin')

True

In [ ]:
ft_index.save(data_path + 'ft_index.bin')

True

In [ ]:
pickle.dump(index_map, open(data_path + 'index_map.pkl', 'wb'))

## Тестирование чат-бота

In [ ]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [ ]:
TEXT = "как научиться читать мысли?"
get_response(TEXT, w2v_index, modelW2V, index_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


['<div> <p>Казалось бы, кому в наше время современных технологий нужно такое умение? Ведь всегда можно взять калькулятор, который имеется в любом мобильном телефоне, и провести необходимые вычисления. Однако иногда нужного устройства может не оказаться рядом. Тут придется полезным умение быстро считать в уме.</p> <p>Существует определенных набор арифметических алгоритмов, каждый из которых в том или ином случае позволяет существенно облегчить счет. Начинать учиться быстрому счету в уме нужно именно с зазубривания этих алгоритмов.</p> <h2>Вычитание 7, 8, 9</h2> <p>Если вам нужно вычесть из любого числа 9, 8 или 7, вычтите из него 10, а затем прибавьте 1, 2 или 3 соответственно. Например: 52-9=52-10+1=43.</p> <h2>Умножение на 9</h2> <p>Для умножения любого числа на 9, умножьте его сначала на 10, то есть просто добавьте ноль в конце, а затем вычтите из полученного результата первые множитель (т.е. то число, которое умножаем).</p> <p>Например, 72*10 (=720) - 72 = 648.</p> <h2>Умножение и д

In [ ]:
TEXT = "как научиться читать мысли?"
get_response(TEXT, ft_index, modelFT, index_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


['<div> <p>Казалось бы, кому в наше время современных технологий нужно такое умение? Ведь всегда можно взять калькулятор, который имеется в любом мобильном телефоне, и провести необходимые вычисления. Однако иногда нужного устройства может не оказаться рядом. Тут придется полезным умение быстро считать в уме.</p> <p>Существует определенных набор арифметических алгоритмов, каждый из которых в том или ином случае позволяет существенно облегчить счет. Начинать учиться быстрому счету в уме нужно именно с зазубривания этих алгоритмов.</p> <h2>Вычитание 7, 8, 9</h2> <p>Если вам нужно вычесть из любого числа 9, 8 или 7, вычтите из него 10, а затем прибавьте 1, 2 или 3 соответственно. Например: 52-9=52-10+1=43.</p> <h2>Умножение на 9</h2> <p>Для умножения любого числа на 9, умножьте его сначала на 10, то есть просто добавьте ноль в конце, а затем вычтите из полученного результата первые множитель (т.е. то число, которое умножаем).</p> <p>Например, 72*10 (=720) - 72 = 648.</p> <h2>Умножение и д

In [ ]:
TEXT = "где счастье?"
get_response(TEXT, w2v_index, modelW2V, index_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


['Это когда у тя в жизни всё в поряде. \n',
 'Просыпаешся утром.Голова болит и настроение казалось бы х....Думаешь: "Че я здесь делаю, в этом городе, где нет возможностей и свободы, надо рвануть куда-н., быть может где-то меня ждет удача,,быть может я найду свою любовь...." Начинаешь вспоминать прожитые дни, прекрасные минуты жизни, друзей бесшабашных, хороших и чертовски красивых подруг, родных, и на душе становится просто КЛ.Идешь на кухню, открываешь холодильник и достаешь оттуда упаковку хорошего прохладного сока, выпиваешь и уже ничего не хочется.Забываешь про дурные мысли и продолжаешь жить.ВОТ ОНО СЧАСТЬЕ!!!. \n',
 'посмотри в зеркало, вот оно. \n']

In [ ]:
TEXT = "где счастье?"
get_response(TEXT, ft_index, modelFT, index_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


['Это когда у тя в жизни всё в поряде. \n',
 'Просыпаешся утром.Голова болит и настроение казалось бы х....Думаешь: "Че я здесь делаю, в этом городе, где нет возможностей и свободы, надо рвануть куда-н., быть может где-то меня ждет удача,,быть может я найду свою любовь...." Начинаешь вспоминать прожитые дни, прекрасные минуты жизни, друзей бесшабашных, хороших и чертовски красивых подруг, родных, и на душе становится просто КЛ.Идешь на кухню, открываешь холодильник и достаешь оттуда упаковку хорошего прохладного сока, выпиваешь и уже ничего не хочется.Забываешь про дурные мысли и продолжаешь жить.ВОТ ОНО СЧАСТЬЕ!!!. \n',
 'посмотри в зеркало, вот оно. \n']